In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/College/F19/CS 4740/NLP_P4

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/College/F19/CS 4740/NLP_P4


In [2]:
import os
import math
import time
import nltk
import random
import spacy
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import init
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from collections import Counter
from tqdm import tqdm
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nltk.download('punkt')
get_pos = spacy.load("en_core_web_sm")
# analyser = SentimentIntensityAnalyzer()
train_data = pd.read_csv('./gpu_train.csv', encoding='latin-1')
dev_data = pd.read_csv('./gpu_dev.csv', encoding='latin-1')
test_data = pd.read_csv('./gpu_test.csv', encoding='latin-1')
data = [train_data, dev_data, test_data]

EMBED_DIM = 32
HIDDEN_DIM = 16
LAYERS = 1
EPOCHS = 1000
LR = 3e-4
DEVICE = torch.device("cuda:0")
NAME = 'POS1'
CURRENT = os.curdir
MODELS = os.path.join(CURRENT, 'experimental_models')
PATH = os.path.join(MODELS, NAME)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# def sentiment_analyzer_scores(sentence):
#     return analyser.polarity_scores(sentence)

In [0]:
list_of_sentences = []
def tag_pos(data):
    pos_counts = Counter()
    pos_lst = []
    for j, df in enumerate(data):
        df_pos = []
        for row in df.iterrows():
            row_pos = []
            for i in range(1, 7):
                parts_of_speech = get_pos(row[1][i])
                sentence_lst = word_tokenize(row[1][i])
                list_of_sentences.append(sentence_lst)
                sentence_pos = []
                count = 0
                iter_pos = iter(parts_of_speech)
                for pos in iter_pos:
                    if pos.pos_ is 'SPACE':
                        continue
                    if '-' in sentence_lst[count] and sentence_lst[count] is not '-':
                        for i in range(sentence_lst[count].count('-')):
                            next(iter_pos)
                            next(iter_pos)
                        sentence_pos.append(pos.pos_)
                        count += 1
                        continue
                    sentence_pos.append(pos.pos_)
                    count += 1
                if not len(sentence_lst) == len(sentence_pos):
                    print(j)
                    print(sentence_lst)
                    print(sentence_pos)
                pos_counts.update(sentence_pos)
                # print(sentence_lst)
                # print(sentence_pos)
                row_pos.append(sentence_pos)
            df_pos.append(row_pos)
        pos_lst.append(df_pos)
    return pos_lst, pos_counts

In [0]:
pos_data = tag_pos(data)

In [0]:
model = Word2Vec(list_of_sentences, size=EMBED_DIM, min_count=1)
name = 'word2vec' + str(EMBED_DIM) + '_gpu.model'
model.save(name)

In [0]:
w2v = 'word2vec' + str(EMBED_DIM) + '_gpu.model'
WORD2VEC = Word2Vec.load(w2v)

In [0]:
def get_one_hot(group, row, sentence, word):
    pos = pos_data[0][group][row][sentence][word]
    return [1 if pos == list(pos_data[1].keys())[i] else 0 for i in range(len(pos_data[1].keys()))]

In [0]:
def embed(train_data, dev_data, test_data):
    training_data = [] 
    for row in train_data.iterrows():
        pos = [row[1][0]]
        neg = [row[1][0]]
        for i in range(1, 5):
            lst = [np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], i-1, j)) \
                   for j, word in enumerate(word_tokenize(row[1][i]))]
            pos.append(lst)
            neg.append(lst)
        if row[1][7] == 1:
            pos.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 4, j)) \
                   for j, word in enumerate(word_tokenize(row[1][5]))])
            neg.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 5, j)) \
                   for j, word in enumerate(word_tokenize(row[1][6]))])
        elif row[1][7] == 2:
            pos.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 5, j)) \
                   for j, word in enumerate(word_tokenize(row[1][6]))])
            neg.append([np.array(list(WORD2VEC.wv[word]) + get_one_hot(0, row[0], 4, j)) \
                   for j, word in enumerate(word_tokenize(row[1][5]))])
        pos.append(1)
        neg.append(0)
        training_data.append(pos)
        training_data.append(neg)
    
    development_data = []
    for row in dev_data.iterrows():
        sentences = [row[1][0]]
        for i in range(1, 7):
            lst = [np.array(list(WORD2VEC.wv[word]) + get_one_hot(1, row[0], i-1, j)) \
                   for j, word in enumerate(word_tokenize(row[1][i]))]
            sentences.append(lst)
        sentences.append(row[1][7] - 1)
        development_data.append(sentences)
        
    testing_data = []
    for row in test_data.iterrows():
        sentences = [row[1][0]]
        for i in range(1, 7):
            lst = [np.array(list(WORD2VEC.wv[word]) + get_one_hot(2, row[0], i-1, j)) \
                   for j, word in enumerate(word_tokenize(row[1][i]))]
            sentences.append(lst)
        testing_data.append(sentences)
        
    return training_data, development_data, testing_data

In [0]:
training_data, development_data, testing_data = embed(data[0], data[1], data[2])

In [0]:
class NSP(nn.Module):
    def __init__(self):
        super(NSP, self).__init__()
        self.beginning = nn.GRU(EMBED_DIM+16, HIDDEN_DIM, LAYERS, batch_first=True, bidirectional=False)
        self.ending = nn.GRU(EMBED_DIM+16, HIDDEN_DIM, LAYERS, batch_first=True, bidirectional=False)
        self.linear = nn.Linear(HIDDEN_DIM, 2)
        self.softmax = nn.LogSoftmax(dim=0)
        self.criterion = nn.NLLLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=LR)
        self.cuda(device=DEVICE)
        
    def setup(self, data):
        input_1 = torch.tensor(np.expand_dims(data[0] + data[1] + data[2] + data[3], axis=0), device=DEVICE, dtype=torch.float)
        input_2 = torch.tensor(np.expand_dims(data[4], axis=0), device=DEVICE, dtype=torch.float)
        return input_1, input_2

    def compute_Loss(self, predicted_vector, gold_label):
        return self.criterion(predicted_vector, gold_label)

    def forward(self, data):
        input_1, input_2 = self.setup(data)
        h_0 = torch.zeros((LAYERS, 1, HIDDEN_DIM), device=DEVICE)
        __, h_n = self.beginning(input_1, h_0)
        output, __ = self.ending(input_2, h_n)
        x = output[0][-1]
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [0]:
print('Initializing Model')
model = NSP()
prev_dev_acc = 0.0
for epoch in range(EPOCHS):
    checkpoint = PATH + '-e' + str((epoch + 1))
    model.train()
    model.optimizer.zero_grad()
    loss = None
    correct = 0
    total = 0
    start_time = time.time()
    print('Training started for epoch {}'.format(epoch + 1))
    random.shuffle(training_data)
    N = len(training_data)
    for index  in tqdm(range(N)):
        model.optimizer.zero_grad()
        sample = training_data[index]
        input_vector = sample[1:6]
        gold_label = sample[6]
        predicted_vector = model(input_vector)
        predicted_label = torch.argmax(predicted_vector)
        correct += int(predicted_label == gold_label)
        total += 1
        loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label], device=DEVICE))
        loss.backward()
        model.optimizer.step()
    print('Training accuracy for epoch {}: {}'.format(epoch + 1, correct / total))
    correct = 0
    total = 0
    start_time = time.time()
    random.shuffle(development_data)
    N = len(development_data)
    model.eval()
    model.optimizer.zero_grad()
    for index in tqdm(range(N)):
        sample = development_data[index]
        input_1 = sample[1:6]
        input_2 = sample[1:5] + [sample[6]]
        gold_label = sample[7]
        prediction_1 = model(input_1)
        prediction_2 = model(input_2)
        prob_truthful_1 = prediction_1[1]
        prob_false_1 = prediction_1[0]
        prob_truthful_2 = prediction_2[1]
        prob_false_2 = prediction_2[0]
        probs = [prob_truthful_1, prob_false_1, prob_truthful_2, prob_false_2]
        max_index = probs.index(max(probs))
        if max_index == 0 or max_index == 3:
            predicted_label = 0
        if max_index == 1 or max_index == 2:
            predicted_label = 1
        correct += int(predicted_label == gold_label)
        total += 1
    dev_acc = correct / total
    if dev_acc > prev_dev_acc and dev_acc > 0.67:
        prev_dev_acc = dev_acc
        print('New Best Accuracy: {}'.format(dev_acc))
        acc = int(100 * dev_acc)
        torch.save(model.state_dict(), checkpoint + '-a' + str(acc) + '.pt')
    print('Development accuracy for epoch {}: {}'.format(epoch + 1, correct / total))

torch.save(model.state_dict(), PATH + '-final.pt')